# Lesson 3: 데이터 패키징

Deeplearning.AI & Upstage의 다음 강의를 듣고 정리한 노트북입니다.

https://learn.deeplearning.ai/courses/pretraining-llms/lesson/4/packaging-data-for-pretraining

이 노트북에서는 LLM 사전 훈련을 위한 데이터를 토크나이징 한 후 적절한 길이로 패킹하는 방법에 대해 학습합니다.

In [5]:
import warnings
warnings.filterwarnings("ignore")

## 토크나이징

이전 레슨에서 저장한 데이터를 불러옵니다. 다만 원할한 학습을 위해 `shard` 메소드를 이용하여 데이터의 1/10 만 사용하도록 합니다.

In [10]:
import datasets 

dataset = datasets.load_dataset(
    "parquet",
    data_files="data/preprocessed_dataset.parquet",
    split="train"
)

In [11]:
dataset = dataset.shard(num_shards=10, index=0)
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 4048
})


토크나이저를 로드합니다.

In [12]:
from transformers import AutoTokenizer
model_path_or_name = "upstage/SOLAR-10.7B-v1.0"
tokenizer = AutoTokenizer.from_pretrained(
    model_path_or_name,
    use_fast=False
)

In [13]:
tokenizer.tokenize("I'm a short sentence") 

['▁I', "'", 'm', '▁a', '▁short', '▁sentence']

토크나이징된 결과에 bos/eos 토큰을 추가하는 함수를 사용합니다.

In [14]:
def tokenization(example):
    tokens = tokenizer.tokenize(example['text'])
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    
    # Add <bos>, <eos> tokens to the front and back of tokens_ids 
    # bos: begin of sequence, eos: end of sequence
    token_ids = [tokenizer.bos_token_id] + token_ids + [tokenizer.eos_token_id]
    example['input_ids'] = token_ids
    example['num_tokens'] = len(token_ids)

    return example

In [15]:
dataset = dataset.map(tokenization, load_from_cache_file=False)
print(dataset)

Map:   0%|          | 0/4048 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'num_tokens'],
    num_rows: 4048
})


In [16]:
sample = dataset[3]

print("text", sample["text"][:30]) # 
print("\ninput_ids", sample["input_ids"][:30])
print("\nnum_tokens", sample["num_tokens"])

text The Colorado Climate Center pr

input_ids [1, 415, 15837, 1366, 3314, 6064, 5312, 430, 19102, 304, 1178, 356, 281, 3928, 28725, 9735, 28713, 28725, 264, 1052, 14455, 4623, 28725, 9390, 1452, 274, 28725, 17268, 28713, 28725]

num_tokens 549


총 토큰의 갯수를 조회합니다.

In [18]:
import numpy as np

np.sum(dataset['num_tokens'])

4887707

## 데이터 패킹

![img](./images/data_packing.png)

데이터를 모두 이어붙인 후 적절한 길이로 자르는 패킹을 수행합니다.

In [19]:
input_ids = np.concatenate(dataset['input_ids'])
print(len(input_ids))

4887707


In [20]:
max_seq_length = 64
total_length = len(input_ids) - len(input_ids) % max_seq_length
print(total_length)

4887680


원하는 길이로 떨어지지 않는 나머지 부분은 제외

In [21]:
input_ids = input_ids[:total_length]
print(input_ids.shape)

(4887680,)


In [22]:
input_ids_reshaped = input_ids.reshape(-1, max_seq_length).astype(np.int32)
print(input_ids_reshaped.shape)

(76370, 64)


In [23]:
type(input_ids_reshaped)

numpy.ndarray

HuggingFace 데이터셋 생성

In [24]:
input_ids_list = input_ids_reshaped.tolist()
packaged_pretrain_dataset = datasets.Dataset.from_dict(
    {'input_ids':input_ids_list}
)

print(packaged_pretrain_dataset)

Dataset({
    features: ['input_ids'],
    num_rows: 76370
})


## 저장

In [27]:
packaged_pretrain_dataset.to_parquet('data/packaged_pretrain_dataset.parquet')

Creating parquet from Arrow format:   0%|          | 0/77 [00:00<?, ?ba/s]

19856200

In [28]:
!ls -alh data

total 250280
drwxr-xr-x  4 emart  staff   128B Jul 22 18:06 .
drwxr-xr-x  9 emart  staff   288B Jul 22 17:52 ..
-rw-r--r--  1 emart  staff    11M Jul 22 18:06 packaged_pretrain_dataset.parquet
-rw-r--r--  1 emart  staff   112M Jul 22 17:51 preprocessed_dataset.parquet
